<a href="https://colab.research.google.com/github/Ankur-Soni14/AMEX_ProductTrack/blob/main/AMEX_Round2_bowlerselection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


First, we create the qualified bowlers dataframe, using the cumulative runs criteria. Then we create 'Bowling_Average' using runs and wicket_count and 'Bowling_Strike_Rate' using balls bowled and wicket_count. Both these parameters are set to blank, for matches with no wickets. These coloumns will be used later in the calculation of the cumulative score. Economy and 4 wicket per innings are also calculated.

In [ ]:
import pandas as pd
import numpy as np

batsman_df = pd.read_excel('/content/gdrive/MyDrive/Amex Round 2/6671501a18c55_round2_input.xlsx', sheet_name=1)
bowler_df = pd.read_excel('/content/gdrive/MyDrive/Amex Round 2/6671501a18c55_round2_input.xlsx', sheet_name=2)
match_df = pd.read_excel('/content/gdrive/MyDrive/Amex Round 2/6671501a18c55_round2_input.xlsx', sheet_name=3)

# Calculate cumulative wickets and merge with bowler_df
cumulative_wickets = bowler_df.groupby('bowler_id')['wicket_count'].sum().reset_index()
cumulative_wickets.columns = ['bowler_id', 'cumulative_wickets']
bowler_df = bowler_df.merge(cumulative_wickets, on='bowler_id') # Merge cumulative wickets into bowler_df

# Calculate strike rate for all bowlers in bowler_df
bowler_df['sr'] = bowler_df['balls_bowled'] / bowler_df['wicket_count']
# Replace the strike rate for wicket_count=0 with NaN
bowler_df.loc[bowler_df['wicket_count'] == 0, 'sr'] = np.nan
# Assign the result of the calculation to the variable 'sr'
sr = bowler_df.groupby('bowler_id')['sr'].mean().reset_index()
sr.columns = ['bowler_id', 'Bowling_Strike_Rate'] # Rename column
bowler_df = bowler_df.merge(sr, on='bowler_id') # Merge into bowler_df

# Calculate average for all bowlers in bowler_df
bowler_df['avg'] = bowler_df['runs'] / bowler_df['wicket_count']
# Replace the average for wicket_count=0 with NaN
bowler_df.loc[bowler_df['wicket_count'] == 0, 'avg'] = np.nan
# Assign the result of the calculation to the variable 'avg'
avg = bowler_df.groupby('bowler_id')['avg'].mean().reset_index()
avg.columns = ['bowler_id', 'Bowling_Average'] # Rename column
bowler_df = bowler_df.merge(avg, on='bowler_id') # Merge into bowler_df

# Calculate Economy and merge with bowler_df
Economy = bowler_df.groupby('bowler_id')['economy'].mean().reset_index()
Economy.columns = ['bowler_id', 'Economy'] # Rename column
bowler_df = bowler_df.merge(Economy, on='bowler_id') # Merge into bowler_df

# Calculate the count of instances where a bowler took 4 wickets
four_wickets_count = bowler_df.groupby('bowler_id')['wicket_count'].apply(lambda x: x[x >= 4].count()).reset_index(name='4W_Per_Innings')
# Merge the count with the bowler_df
bowler_df = bowler_df.merge(four_wickets_count, on='bowler_id')

# Filter for qualified bowlers and keep only one entry per bowler_id
qualified_bowlers = bowler_df[bowler_df['cumulative_wickets'] > 10]

#Calculate matches played by each qualified bowler
matches_per_player = qualified_bowlers['bowler_id'].value_counts()
# average
average_matches = matches_per_player.mean()
# Find the 25th percentile
quarter = matches_per_player.quantile(0.25)

print("25th Percentile: ", quarter)
print(f"The average number of matches played by a player is: {average_matches:.2f}")

qualified_bowlers = qualified_bowlers.drop_duplicates(subset='bowler_id', keep='first')
qualified_bowlers = qualified_bowlers.sort_values(by='cumulative_wickets', ascending=False)

# Print the qualified bowlers DataFrame
print("Qualified Bowlers (cumulative wickets > 10, one entry per bowler_id):")
print(qualified_bowlers[['bowler_id', 'cumulative_wickets', 'Bowling_Strike_Rate', 'Bowling_Average', 'Economy', '4W_Per_Innings']]) # Display only the required columns

25th Percentile:  16.0
The average number of matches played by a player is: 30.27
Qualified Bowlers (cumulative wickets > 10, one entry per bowler_id):
       bowler_id  cumulative_wickets  Bowling_Strike_Rate  Bowling_Average  \
308      5554254                 175            16.925680        18.191327   
1828     8131255                 118            15.968905        21.213930   
828      5788418                 108            14.319940        18.592262   
11041    5554142                 106            15.530055        21.653005   
13575    1613919                 106            17.450980        18.569853   
...          ...                 ...                  ...              ...   
4639     4160267                  11            15.750000        22.229167   
15175    3056752                  11            11.666667        16.666667   
4727     4874092                  11             8.687500        12.833333   
16253    8766435                  11            11.500000         7.

Now, the total points for each bowler is calculated using the points awarding system. These points have only been calculated for bowler who have played more than the 25th percentile of matches played by each bowler. This enables us to discard players who have played too few matches as points here may lead to misleading results.

In [ ]:
import pandas as pd

# Assuming you have the qualified_bowlers DataFrame from the previous steps

def calculate_bowler_score(row):
    strike_rate_score = 0
    economy_score = 0
    avg_score = 0
    fourw_per_innings_score = 0

    n = matches_per_player.get(row['bowler_id'], 0)
    if n>16:

      # Strike Rate score
      if row['Bowling_Strike_Rate'] <= 15: # Access column using string
          strike_rate_score = 30
      elif 15 < row['Bowling_Strike_Rate'] <= 19: # Access column using string
          strike_rate_score = 20
      elif 19 < row['Bowling_Strike_Rate'] <= 24: # Access column using string
          strike_rate_score = 10
      elif row['Bowling_Strike_Rate'] > 24: # Access column using string
          strike_rate_score = 0

      # Economy score
      if row['Economy'] <= 3: # Access column using string
          economy_score = 50
      elif 3 < row['Economy'] <= 5: # Access column using string
          economy_score = 40
      elif 5 < row['Economy'] < 7: # Access column using string
          economy_score = 30
      elif row['Economy'] >= 7: # Access column using string
          economy_score = 0


      # Average score - Use 'Bowling_Average' instead of 'avg'
      if row['Bowling_Average'] <= 20:
          avg_score = 30
      elif 20 < row['Bowling_Average'] <= 30:
          avg_score = 20
      elif 30 < row['Bowling_Average'] <= 40:
          avg_score = 10
      elif row['Bowling_Average'] > 40:
          avg_score = 0


      # 4W_Per_Innings score
      if row['4W_Per_Innings'] >= 4:
          fourw_per_innings_score = 30
      elif row['4W_Per_Innings'] == 3:
          fourw_per_innings_score = 20
      elif row['4W_Per_Innings'] == 2:
          fourw_per_innings_score = 20
      elif row['4W_Per_Innings'] == 1:
          fourw_per_innings_score = 10
      elif row['4W_Per_Innings'] == 0:
          fourw_per_innings_score = 0

      total_points = strike_rate_score + economy_score + avg_score  + fourw_per_innings_score
      return total_points

    else:
      return 0

# Apply the scoring function to each row and add a new column 'total_score'
qualified_bowlers['bowler_points'] = qualified_bowlers.apply(calculate_bowler_score, axis=1)

# Drop the existing 'bowler_points' column from bowler_df if it exists
if 'bowler_points' in bowler_df.columns:
    bowler_df = bowler_df.drop('bowler_points', axis=1)

# Use a left merge to keep all rows of bowler_df, but only bring 'bowler_points' from qualified_bowlers
bowler_df = bowler_df.merge(qualified_bowlers[['bowler_id', 'bowler_points']], on='bowler_id', how='left')

# Calculate the cumulative score for all qualified batsmen
qualified_bowlers = qualified_bowlers.sort_values(by='bowler_points', ascending=False)
print(qualified_bowlers[['bowler_id','bowler_points']].to_string())

       bowler_id  bowler_points
308      5554254            110
997        49496            100
2785     3519011            100
3498     3566240            100
14441    3715697            100
10384    3298427            100
4740      328607            100
7769     7783432             90
14343    2018001             90
725      3200756             90
4376     3373866             90
9189      284339             90
13466    6347494             90
7289     7341186             90
13822    4377694             90
15013    7968225             90
14007     393014             90
13575    1613919             90
828      5788418             90
8819      305143             90
7987     3759846             80
16176    8118914             80
7040     8180500             80
4180     1963443             80
10697    5404440             80
10539     334963             80
10787    3245689             80
12941    3685100             80
4897     3062079             80
10262    8055417             80
11929   

Required bowler_id are extracted from nearbest_playing11.

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/gdrive/MyDrive/Amex Round 2/6671502b6a89e_nearbest_playing11.csv')

# Specify the column name
column_name = 'Player_id'

# Extract the specified column as an array
column_data = df[column_name].to_numpy()

# Print the extracted column data
print(column_data)

[ 325814 1626526 1506098   82228 8250115 4171460 2275195 5652758 8986193
 8246573 7758638]


In [ ]:
nearbestplaying11_df = pd.read_csv('/content/gdrive/MyDrive/Amex Round 2/6671502b6a89e_nearbest_playing11.csv')

# Specify the bowler IDs you want to extract
bowler_ids = [3298427,
3519011,
5554254,
49496,
328607,
3715697]

# Extract the rows for the specified bowler IDs, including 'bowler_points'
extracted_rows = qualified_bowlers[qualified_bowlers['bowler_id'].isin(bowler_ids)][['bowler_id', 'bowler_points', 'cumulative_wickets', 'Bowling_Strike_Rate', 'Bowling_Average', 'Economy', '4W_Per_Innings']]

print(extracted_rows.to_string()) # No need to specify columns again, as they are already selected

       bowler_id  bowler_points  cumulative_wickets  Bowling_Strike_Rate  Bowling_Average   Economy  4W_Per_Innings
308      5554254            110                 175            16.925680        18.191327  6.815938               6
997        49496            100                  31            16.156250        16.104167  6.203043               3
4740      328607            100                  27            15.095238        14.059524  5.811200               2
2785     3519011            100                  76            16.456522        17.465580  6.692174               2
14441    3715697            100                  64            18.642857        18.728175  6.739138               2
10384    3298427            100                  23            10.636364         9.750000  6.833043               1


Explanation for recency and consistency score already given in batsman file. Similar point system as the one given is used for recency and consistency, so that these factors are not unidimensional. Instead of '4W per Innings', wicket_count has been used.

Once again, like with batsmen, weights (used in recency score) are only given to players with more than average no. of matches; otherwise recent matches will take up a huge fraction of matches played.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def calculate_match_score(row):
    strike_rate_score = 0
    economy_score = 0
    avg_score = 0
    wickets_score = 0

    # Strike Rate score
    if row['sr'] <= 15: # Access column using string
          strike_rate_score = 30
    elif 15 < row['sr'] <= 19: # Access column using string
          strike_rate_score = 20
    elif 19 < row['sr'] <= 24: # Access column using string
          strike_rate_score = 10
    elif row['sr'] > 24: # Access column using string
          strike_rate_score = 0

    # Economy score
    if row['economy'] <= 3: # Access column using string
        economy_score = 50
    elif 3 < row['economy'] <= 5: # Access column using string
        economy_score = 40
    elif 5 < row['economy'] < 7: # Access column using string
        economy_score = 30
    elif row['economy'] >= 7: # Access column using string
        economy_score = 0


    # Average score - Use 'Bowling_Average' instead of 'avg'
    if row['avg'] <= 20: # Use the correct column name 'avg'
        avg_score = 30
    elif 20 < row['avg'] <= 30:
        avg_score = 20
    elif 30 < row['avg'] <= 40:
        avg_score = 10
    elif row['avg'] > 40:
        avg_score = 0


    # wickets score
    if row['wicket_count'] >= 4:
        wickets_score = 50
    elif row['wicket_count'] == 3:
        wickets_score = 30
    elif row['wicket_count'] == 2:
        wickets_score = 20
    elif row['wicket_count'] == 1:
        wickets_score = 10
    elif row['wicket_count'] == 0:
        wickets_score = 0

    total_score = strike_rate_score + avg_score + economy_score + wickets_score
    return total_score

def calculate_weighted_consistency_score(match_scores, bowler_points):
    if len(match_scores) == 0 or bowler_points == 0:
        return 0

    # Calculate the coefficient of variation (CV)
    cv = np.std(match_scores) / (np.mean(match_scores) + 1e-6)

    # Calculate the average match score
    avg_match_score = np.mean(match_scores)

    # Calculate the raw weighted consistency score
    raw_score = (1 / (cv + 1)) * avg_match_score

    # Normalize the raw score to be a small fraction of bowler_points
    max_tiebreak_contribution = bowler_points * 0.1  # 10% of bowler_points
    normalized_score = (raw_score / np.max(match_scores)) * max_tiebreak_contribution

    return normalized_score

def calculate_tiebreak_score(match_scores, bowler_points):
    if bowler_points == 0:
        return 0
    # Recency score using linear weighting
    n = len(match_scores)
    if n>30:
      recent_matches = 5  # Consider last 5 matches or half, whichever is larger
      weights = np.ones(n)
      weights[:-recent_matches] = 0.5

      # Normalize weights
      weights = weights / np.sum(weights)

    else:
      recent_matches=0
      weights = np.ones(n)
      weights[:-recent_matches] = 0.5
      # Normalize weights
      weights = weights / np.sum(weights)

    # Recency score
    recency_score = np.average(match_scores, weights=weights)

    # Consistency score
    consistency_score = calculate_weighted_consistency_score(match_scores, bowler_points)

    # Tiebreak score (1/3 weight each, which will eventually be added to 1/3 of batsman_points, so that points, recency, consistency - all have equal weight)
    return recency_score/3 + consistency_score/3

def process_bowler_data(bowler_df):
    # Sort the DataFrame by match_date for each bowler
    bowler_df = bowler_df.sort_values(['bowler_id', 'match_dt'])

    # Calculate match score for each match
    bowler_df['match_score'] = bowler_df.apply(calculate_match_score, axis=1)

    # Calculate tiebreak score for each bowler
    # Pass bowler_points to calculate_tiebreak_score
    tiebreak_scores = bowler_df.groupby('bowler_id').apply(lambda x: calculate_tiebreak_score(x['match_score'], x['bowler_points'].iloc[0]))

    return bowler_df, tiebreak_scores

# Process the bowler data
processed_bowler_df, tiebreak_scores = process_bowler_data(bowler_df)

# Add tiebreak scores to qualified_bowlers DataFrame
qualified_bowlers['tiebreak_score'] = qualified_bowlers['bowler_id'].map(tiebreak_scores)

# Combined score
qualified_bowlers['combined_score'] = qualified_bowlers['bowler_points']/3 + qualified_bowlers['tiebreak_score']

# Sort qualified_bowlers by tiebreak_score
qualified_bowlers = qualified_bowlers.sort_values('combined_score', ascending=False)

print("Qualified bowlers with tiebreak scores:")
print(qualified_bowlers[['bowler_id', 'bowler_points', 'tiebreak_score', 'combined_score']].to_string())

<ipython-input-5-91679338d0ac>:73: RuntimeWarning: invalid value encountered in scalar divide
  normalized_score = (raw_score / np.max(match_scores)) * max_tiebreak_contribution


Qualified bowlers with tiebreak scores:
       bowler_id  bowler_points  tiebreak_score  combined_score
308      5554254            110       23.073064       59.739731
997        49496            100       24.109929       57.443263
4740      328607            100       22.865971       56.199304
2785     3519011            100       22.443645       55.776978
3498     3566240            100       21.565093       54.898427
4376     3373866             90       24.732673       54.732673
14441    3715697            100       21.283152       54.616486
7289     7341186             90       24.269740       54.269740
9189      284339             90       23.738778       53.738778
14343    2018001             90       23.647364       53.647364
10384    3298427            100       20.154542       53.487876
13822    4377694             90       23.143086       53.143086
7769     7783432             90       22.853425       52.853425
18443    4533682             80       25.652395       52.319061
